In [105]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import fiona
from shapely.geometry import shape, mapping, Point, Polygon, MultiPolygon

# load dataframe
df = pd.read_csv('data/ovitrap_data.csv', index_col=0)
df.date = pd.to_datetime(df.date)

print(df.id.nunique(), df.name.nunique(), df.latitude.nunique(), df.longitude.nunique())
df.head()

12306 10863 11570 11652


,date,id,latitude,longitude,name,value
0,2012-06-19,3,14.5044,121.056,Kapitan Jose Cardones Memorial School,17.31
1,2012-07-03,3,14.5044,121.056,Kapitan Jose Cardones Memorial School,15.52
2,2012-07-17,3,14.5044,121.056,Kapitan Jose Cardones Memorial School,30.00
3,2012-07-24,3,14.5044,121.056,Kapitan Jose Cardones Memorial School,22.64
4,2012-09-11,3,14.5044,121.056,Kapitan Jose Cardones Memorial School,28.13


In [118]:
# remove locations outside the philippines, aggregate per id
df_per_id = df.drop(columns=['date', 'value', 'name'])
df_per_id = df_per_id[df_per_id['latitude'] < 30.]
df_per_id = df_per_id[df_per_id['latitude'] > 0.]
df_per_id = df_per_id[df_per_id['longitude'] < 130.]
df_per_id = df_per_id[df_per_id['longitude'] > 110.]
df_per_id = df_per_id.groupby('id').median()
print(df_per_id.index.nunique(), df_per_id.latitude.nunique(), df_per_id.longitude.nunique())
df_per_id.head()

# save shapefile with ovitrap locations
gdf_ovi = gpd.GeoDataFrame(df_per_id,
                           geometry=gpd.points_from_xy(df_per_id.longitude, df_per_id.latitude),
                           crs={'init': 'epsg:4326'})
gdf_ovi = gdf_ovi.drop(columns=['latitude', 'longitude'])
gdf_ovi.to_file("data/ovitrap_locations.shp")

12289 11553 11635


,latitude,longitude
id,,
2,14.51108,121.05721
3,14.50440,121.05600
6,14.50036,121.06263
7,14.50152,121.05044
8,14.52552,121.07533


In [4]:
# load admin levels
admin_level = 1
admin_shapefile = '/home/jmargutti/Host/rodekruis/ERA/shapefiles/phl_admbnda_adm'+str(admin_level)+'_psa_namria_20180130.shp'
gdf_adm = gpd.read_file(admin_shapefile)
gdf_adm.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,53.623477,1.050272,Autonomous Region in Muslim Mindanao,PH150000000,None,ARMM,None,Philippines (the),PH,2016-06-15,2018-01-30,None,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4..."
1,8.027454,1.546712,Cordillera Administrative Region,PH140000000,None,CAR,None,Philippines (the),PH,2016-06-15,2018-01-30,None,"POLYGON ((121.22208 18.50058, 121.22086 18.483..."
2,2.320234,0.050216,National Capital Region,PH130000000,None,NCR,None,Philippines (the),PH,2016-06-15,2018-01-30,None,"POLYGON ((121.03842 14.78525, 121.03876 14.785..."
3,11.231749,1.055027,Negros Island Region,PH180000000,None,NIR,None,Philippines (the),PH,2016-06-15,2018-01-30,None,"MULTIPOLYGON (((123.27039 9.08460, 123.27040 9..."
4,14.995101,1.043983,Region I,PH010000000,None,Ilocos Region,None,Philippines (the),PH,2016-06-15,2018-01-30,None,"MULTIPOLYGON (((119.86596 15.81539, 119.86597 ..."


In [ ]:
# plot locations over admin levels
fig, ax = plt.subplots(figsize=(15,15))
gdf_adm.plot(ax=ax, facecolor='gray')
gdf_ovi.plot(ax=ax, color='blue', markersize=5)
plt.tight_layout()

In [ ]:
# prepare dataframe with admin level
df_per_id_with_admin = df_per_id.copy()
df_per_id_with_admin['adm'] = 'nan'

# load shapefiles with ovitrap locations
points = ([pt for pt in fiona.open("data/ovitrap_locations.shp")])

# convert admin levels into a dictionary
multipol = fiona.open(admin_shapefile)
admin_bound = {}
for n, multi in enumerate(multipol):
    admin_bound[multi['properties']['ADM'+str(admin_level)+'_EN']] = shape(multi['geometry'])

# loop over polygons amd points, assign admin level to dataframe
for i, pt in enumerate(tqdm(points)):
    coords = pt['geometry']['coordinates']
    point = shape(pt['geometry'])
    id = pt['properties']['id']
    if not df_per_id_with_admin.loc[id, 'adm'] == 'nan':
        continue
    for name, polygon in admin_bound.items():
        if point.within(polygon):
            df_per_id_with_admin.at[id, 'adm'] = name
            break
    if df_per_id_with_admin.loc[id, 'adm'] == 'nan':
        print('WARNING: point', id, coords, 'not found !!!!')
    if i % 1000 == 0:
        df_per_id_with_admin.to_csv('data/ovitrap_locations_adm'+str(admin_level)'.csv')

print(df_per_id_with_admin.head())
# save dataframe with ovitrap locations and admin levels
df_per_id_with_admin.to_csv('data/ovitrap_locations_adm'+str(admin_level)'.csv')

In [109]:
# add admin level to original ovitrap data

import numpy as np
def add_adm(x):
    try:
        return df_per_id_with_admin.loc[x, 'adm']
    except:
        return np.nan

df_with_adm = df.copy()
df_with_adm['adm'] = df_with_adm['id'].apply(add_adm)
df_with_adm = df_with_adm.dropna(subset=['adm'])
df_with_adm.index = df_with_adm.date
df_with_adm = df_with_adm.drop(columns=['date'])
df_with_adm = df_with_adm.sort_index()

df_with_adm.head()

,id,latitude,longitude,name,value,adm
date,,,,,,
2012-01-21,2606,10.667693,122.945766,Educ.&amp; Training Center Sch.III,52.00,Negros Island Region
2012-01-28,2566,10.646268,122.930660,Don Jose R. Torres Elementary School,3.00,Negros Island Region
2012-06-11,6246,9.531780,123.306420,Samboan Central Elementary School,26.09,Region VII
2012-06-15,924,16.899172,121.707487,Nagrumbuan Elementary School,100.00,Region II
2012-06-19,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,17.31,National Capital Region


In [110]:
# aggregate ovitrap data per admin level and month

# prepare final dataframe (dates, admin levels)
adm_levels = df_.adm.unique()
months = pd.date_range(start='1/1/2012', end='12/1/2019', freq='MS') 
df_final = pd.DataFrame(index=pd.MultiIndex.from_product([adm_levels, months],
                                                         names=['adm_level', 'date']))
df_final['count_ovi'] = 0.
df_final['sum_ovi'] = 0.
df_final['sum_sq_ovi'] = 0.
df_final.tail()

count_ovi  sum_ovi  \
adm_level                            date                             
Autonomous Region in Muslim Mindanao 2019-08-01        0.0      0.0   
                                     2019-09-01        0.0      0.0   
                                     2019-10-01        0.0      0.0   
                                     2019-11-01        0.0      0.0   
                                     2019-12-01        0.0      0.0   

                                                 sum_sq_ovi  
adm_level                            date                    
Autonomous Region in Muslim Mindanao 2019-08-01         0.0  
                                     2019-09-01         0.0  
                                     2019-10-01         0.0  
                                     2019-11-01         0.0  
                                     2019-12-01         0.0

In [111]:
from tqdm import tqdm
tqdm.pandas(desc="progress")
from tqdm import tqdm_notebook

df_ovi = df_.reset_index()
            
def aggregate_ovi(row):
    
    date = row['date'].replace(day=1)
    df_final.at[(row['adm'], date), 'count_ovi'] += 1
    df_final.at[(row['adm'], date), 'sum_ovi'] += row['value']
    df_final.at[(row['adm'], date), 'sum_sq_ovi'] += pow(row['value'],2.)
        
df_ovi.progress_apply(aggregate_ovi, axis=1)

df_final.head()


progress:  73%|███████▎  | 100492/137200 [00:24<00:08, 4196.40it/s]


progress: 100%|██████████| 137200/137200 [00:32<00:00, 4220.57it/s]

count_ovi  sum_ovi  sum_sq_ovi
adm_level            date                                      
Negros Island Region 2012-01-01        2.0     55.0      2713.0
                     2012-02-01        0.0      0.0         0.0
                     2012-03-01        0.0      0.0         0.0
                     2012-04-01        0.0      0.0         0.0
                     2012-05-01        0.0      0.0         0.0

In [112]:
translate = {'Negros Island Region': 'NIR',
             'Region VII': 'VII',
             'Region II': 'II',
             'National Capital Region': 'NCR',
             'Region XI': 'XI',
             'Region III': 'III',
             'Region IX': 'IX',
             'Cordillera Administrative Region': 'CAR',
             'Region VI': 'VI',
             'Region I': 'I',
             'Region V': 'V',
             'Region X': 'X',
             'Region IV-B': 'IV-B',
             'Region XII': 'XII',
             'Region VIII': 'VIII',
             'Region XIII': 'CARAGA',
             'Region IV-A': 'IV-A',
             'Autonomous Region in Muslim Mindanao': 'ARMM'
            }
df_final = df_final.rename(index=translate)
df_final.head()

count_ovi  sum_ovi  sum_sq_ovi
adm_level date                                      
NIR       2012-01-01        2.0     55.0      2713.0
          2012-02-01        0.0      0.0         0.0
          2012-03-01        0.0      0.0         0.0
          2012-04-01        0.0      0.0         0.0
          2012-05-01        0.0      0.0         0.0

In [114]:
# calculate mean, error and relative error
df_final = df_final.assign(mean_ovi=lambda x: x.sum_ovi / x.count_ovi)
df_final = df_final.assign(error_ovi=lambda x: pow(((x.sum_sq_ovi / x.count_ovi) - pow(x.sum_ovi / x.count_ovi, 2.)) / x.count_ovi, 0.5))
df_final = df_final.assign(error_relative_ovi=lambda x: x.error_ovi / x.mean_ovi)

df_final = df_final.drop(columns=['sum_ovi', 'sum_sq_ovi'])

df_final.to_csv('data/ovitrap_data_month_adm'+str(admin_level)+'.csv')

df_final.head()

count_ovi  mean_ovi  error_ovi  error_relative_ovi
adm_level date                                                          
NIR       2012-01-01        2.0      27.5  17.324116            0.629968
          2012-02-01        0.0       NaN        NaN                 NaN
          2012-03-01        0.0       NaN        NaN                 NaN
          2012-04-01        0.0       NaN        NaN                 NaN
          2012-05-01        0.0       NaN        NaN                 NaN